In [3]:
!pip install flask

Defaulting to user installation because normal site-packages is not writeable
  Using cached flask-3.0.3-py3-none-any.whl.metadata (3.2 kB)
  Using cached blinker-1.8.2-py3-none-any.whl.metadata (1.6 kB)
Using cached flask-3.0.3-py3-none-any.whl (101 kB)
Using cached blinker-1.8.2-py3-none-any.whl (9.5 kB)


In [4]:

from flask import Flask, request, jsonify

In [2]:

import cv2
import face_recognition
import numpy as np
import base64

In [12]:
app = Flask(__name__)

# Load known face encodings and names
Modi_image = face_recognition.load_image_file("Modi.jpg")
Modi_face_encoding = face_recognition.face_encodings(Modi_image)[0]

Kohli_image = face_recognition.load_image_file("Kohli.jpg")
Kohli_face_encoding = face_recognition.face_encodings(Kohli_image)[0]

Nihar_image = face_recognition.load_image_file("Nihar.jpg")
Nihar_face_encoding = face_recognition.face_encodings(Nihar_image)[0]

known_face_encodings = [
    Modi_face_encoding,
    Kohli_face_encoding,
    Nihar_face_encoding
]
known_face_names = [
    "Modi",
    "Kohli",
    "Nihar"
]
aadhaar_details = {
    "Modi": "Aadhaar Details for Modi: [1234-5678-9101]",
    "Kohli": "Aadhaar Details for Kohli: [2345-6789-1012]",
    "Nihar": "Aadhaar Details for Nihar: [3456-7890-1123]"
}
recognized_faces = set()

In [13]:

@app.route('/process', methods=['POST'])
def process_frame():
    data = request.json
    frame_data = data['frame']
    frame = base64.b64decode(frame_data)
    np_frame = np.frombuffer(frame, dtype=np.uint8)
    img = cv2.imdecode(np_frame, flags=1)

    small_frame = cv2.resize(img, (0, 0), fx=0.25, fy=0.25)
    rgb_small_frame = small_frame[:, :, ::-1]
    
    face_locations = face_recognition.face_locations(rgb_small_frame)
    face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

    face_names = []
    for face_encoding in face_encodings:
        matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
        name = "Unknown"

        face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
        best_match_index = np.argmin(face_distances)
        if matches[best_match_index]:
            name = known_face_names[best_match_index]

        face_names.append(name)

        if name in aadhaar_details and name not in recognized_faces:
            print(aadhaar_details[name])
            recognized_faces.add(name)

    response = {
        "face_names": face_names,
        "face_locations": face_locations
    }
    return jsonify(response)

In [14]:
if __name__ == '__main__':
    app.run(debug=True, port=5001)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5001
Press CTRL+C to quit
 * Restarting with stat


SystemExit: 1